In [ ]:

# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Imagen 3 Image Generation

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/diyasini13/Image_Customization/blob/main/Demo_Image_Customization%5Bworking_with_2_ref_images%5D.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/diyasini13/Image_Customization/blob/main/Demo_Image_Customization%5Bworking_with_2_ref_images%5D.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>



## Get started


### Install Vertex AI SDK for Python




In [ ]:
%pip install --upgrade --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 28.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Authenticate your notebook environment

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.


In [ ]:
!gcloud config get-value core/project
!gcloud config set project genail300

!gcloud auth application-default login

(unset)
Updated property [core/project].
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=rixGbmXlbQTsBM2AwAy3QUUi0BgBh4&prompt=consent&token_usage=remote&access_type=offline&code_challenge=7EL6l9QMY4bjZlIHASHfuTkH111DzOe2IaQ11sz7dRE&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0ASVgi3LaQwlCqRSGvGIDSO9vTlYRGAcM6m68qfNB-wC1IYym1iVTdbu6Ml4jGxXUxSToxg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests 

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import base64
import io
from typing import Optional, Tuple, Union, List
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as im
import os
import gradio as gr
from vertexai.preview.vision_models import (
    Image,
    ImageGenerationModel,
    StyleReferenceImage,
    SubjectReferenceImage,
)
from vertexai.generative_models import GenerativeModel, Part
from vertexai.generative_models import Image as image_gen

### Generate Images

In [ ]:
def pil_to_image(image: im) -> str:
    """Converts a PIL Image object to a base64 string."""
    with io.BytesIO() as buffer:
        image.save(buffer, format="JPEG")
        img_bytes = buffer.getvalue()
    return img_bytes

In [ ]:
def resize_image(image_path, base_width=300):
  """
  Resizes an image to the specified base width while maintaining aspect ratio.

  Args:
    image_path: Path to the image.
    base_width: Desired width for the image. Defaults to 300.

  Returns:
    The resized Image object.
  """

  def calculate_new_dimensions(image_path, base_width):
    wpercent = (base_width / float(image_path.size[0]))
    hsize = int((float(image_path.size[1]) * float(wpercent)))
    return (base_width, hsize)


  img = image_path.resize(calculate_new_dimensions(image_path, base_width), im.LANCZOS)

  return img

In [ ]:
def generate_image(image1_path, image2_path, desc1, desc2, prompt):
    """Generates an image based on two reference images, their descriptions, and a prompt.

    Args:
        image1_path: The path to the first reference image file.
        image2_path: The path to the second reference image file.
        desc1: Description of the first reference image.
        desc2: Description of the second reference image.
        prompt: The prompt for image generation.

    Returns:
        The generated image as a PIL Image object.
    """
    print(image1_path, image2_path, desc1, desc2, prompt)

    try:
        # Load images from file paths
        # image1 = im.open(image1_path)
        # image2 = im.open(image2_path)


        # base_width = 300
        # wpercent = (base_width / float(image1_path.size[0]))
        # hsize = int((float(image1_path.size[1]) * float(wpercent)))

        img1 = resize_image(image1_path)
        # wpercent = (base_width / float(image2_path.size[0]))
        # hsize = int((float(image2_path.size[1]) * float(wpercent)))

        # img2 = image2_path.resize((base_width, hsize),im.LANCZOS)
        img2= resize_image(image2_path)
        # img3= resize_image(image3_path)

        image1= Image(pil_to_image(img1))
        image2= Image(pil_to_image(img2))
        # image3= Image(pil_to_image(img3))

        print(image1, image2)

        # Create SubjectReferenceImage objects
        ref_image_1 = SubjectReferenceImage(
            image=image1,
            reference_id=1,
            subject_description=desc1,
            subject_type="product",
        )

        ref_image_2 = SubjectReferenceImage(
            image=image2,
            reference_id=2,
            subject_description=desc2,
            subject_type="product",
        )

        # ref_image_3 = SubjectReferenceImage(
        #     image=image3,
        #     reference_id=3,
        #     subject_description=desc3,
        #     subject_type="product",
        # )


        # Generate the image using the customization model
        generation_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")
        negative_prompt = "disfigured, distorted, mutilated, deformed, grotesque, changes to outfit, alterations to clothing, more than 2 hands, more than 2 legs , hallucination" # Customize as needed

        customization_model = ImageGenerationModel.from_pretrained("imagen-3.0-capability-001")
        images = customization_model._generate_images(
            prompt=prompt,
            number_of_images=4,
            negative_prompt=negative_prompt,
            aspect_ratio="1:1",
            reference_images=[ref_image_1, ref_image_2],
            safety_filter_level="block_few",
            person_generation="allow_adult",

        )

        # generated_image = images[0]
        print("Images", images)
        outputs = [this_image._pil_image for this_image in images]
        for i in range(4):
          if len(outputs) < 4 :
            outputs.append(outputs[0])
            print("length:" , len(outputs))
        print("Output",outputs)
        return outputs
        # return generated_image._pil_image



    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
def image_to_bytes(image: im) -> str:
    """Converts a PIL object to a base64 string."""
    with io.BytesIO() as buffer:
        image.save(buffer, format="JPEG")
        img_bytes = buffer.getvalue()
    return img_bytes

In [ ]:
def pil_to_file(image: im, file_path: str) -> str:
    """
    Saves a PIL Image object to the specified file path.

    Args:
        image: The PIL Image object to save.
        file_path: The file path where the image will be saved.

    Returns:
        The file path where the image was saved.
    """
    try:
        image.save(file_path, format="JPEG")  # Change format if needed (e.g., PNG)
        return file_path
    except Exception as e:
        print(f"Error saving image to file: {e}")
        return None


### Generate Descriptions

In [ ]:
def generate_description(image_path):
    """Generates a description for an image using Gemini."""
    try:
        image = resize_image(image_path)
        # new_image_path = pil_to_file(image,"/content/sample_data/image.jpeg")
        image = image_gen.from_bytes(image_to_bytes(image) )# Convert to Vertex AI Image object
        # image = image_to_base64(image)
        # image = Image(pil_to_image(image))
        print(" BS ",image)
        # print("File path", new_image_path)
        model = GenerativeModel("gemini-1.5-flash-002")

        response = model.generate_content(
    [
        image,
        "what is in the image?",
    ]
)

        # Correct prompt format
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred during description generation: {e}") # Print the detailed error
        return "Error generating description."


### Refine Prompt

In [ ]:
def refine_prompt(prompt,desc1,desc2):
    """Refines the given prompt using Gemini."""
    try:
        model = GenerativeModel("gemini-1.5-flash-002")  # Or a suitable Gemini model
        response = model.generate_content(f"""Rewrite the following prompt for image generation, focusing on creating a concise and effective prompt for a fashion catalog image of a model wearing the outfit. Give only one descriptive paragraph answer:
        Prompt: {prompt}
        Description 1: {desc1}
        Description 2: {desc2}
        """)
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred during prompt refinement: {e}")
        return "Error refining prompt."

### Generate Prompt

In [ ]:
def generate_prompt(desc1, desc2):
    """Generates a prompt for image generation using Gemini, based on two descriptions."""
    try:
        model = GenerativeModel("gemini-1.5-flash-002") # Or any appropriate Gemini model
        response = model.generate_content(
            f"""Create one image generation prompt based on these descriptions:
            Description 1: {desc1}
            Description 2: {desc2}


            The prompt should focus on a fashion model  showcasing this particular outfit in different poses and angles highlighting the outfit [1][2].  Be detailed and descriptive, including information about pose, setting, lighting, and overall mood.  Aim for a high-quality, visually appealing image keeping the outfit as described."""
        )
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred during prompt generation: {e}")
        return "Error generating prompt."

In [ ]:
!pip install markdown2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
import markdown2
def generate_style_and_convert_to_markdown(desc1, desc2, prompt):
        try:
            model = GenerativeModel("gemini-1.5-flash-002")
            response = model.generate_content(
                f"""Suggest different ways to style outfits based on these descriptions and the prompt:
                Description 1: {desc1}
                Description 2: {desc2}

                Prompt: {prompt}

                Focus on giving maximum of two providing practical and fashionable styling advice, considering different occasions, accessories, and complementary garments."""
            )

            styling_suggestions_text = response.text.strip()
            html_output = markdown2.markdown(styling_suggestions_text)

            return html_output, gr.update(visible=True) # Update Markdown section visibility
        except Exception as e:
            print(f"An error occurred generating styling suggestions: {e}")
            return "Error generating styling suggestions.", gr.update(visible=False)

### Generate Product Description

In [ ]:

def generate_product_description(image_path):
    """Generates a product description for an image using Gemini."""
    try:
        image = resize_image(image_path)
        # new_image_path = pil_to_file(image,"/content/sample_data/image.jpeg")
        image = image_gen.from_bytes(image_to_bytes(image) )# Convert to Vertex AI Image object
        # image = image_to_base64(image)
        # image = Image(pil_to_image(image))

        model = GenerativeModel("gemini-1.5-flash-002")  # Or a suitable Gemini model

        prompt = (
    [
        image,
        """Generate a product description in a table format after analysing the  image:
        The table should include these details, If a detail is not present in the image, leave it blank:

        | Pattern | |
        | Occasion | |
        | Fabric | |
        | Net Quantity | |
        | Colour | |
        | Product Category | |


        """
        ])

        response = model.generate_content(prompt)
        return response.text.strip()

    except Exception as e:
        print(f"An error occurred generating product description: {e}")
        return "Error generating product description."

###**Campaign** *Generation*



#generate email

In [ ]:
def generate_email(prompt):
    """Generates a fun and sassy email using Gemini based on the image and prompt."""
    try:
        # image = resize_image(image_path)
        # # new_image_path = pil_to_file(image,"/content/sample_data/image.jpeg")
        # # image = image_gen.from_bytes(image_to_bytes(image) )

        # # Convert the image to base64
        # image = image_to_base64(image)

        # print("Email Image: " , image)
        model = GenerativeModel("gemini-1.5-flash-002")

        # Gemini prompt for email generation
        email_prompt = f"""
        Write a fun and sassy email announcing a new collection, based on the uploaded image and the following prompt:
        {prompt}



        The email should be engaging and reflect current fashion trends. Target a young, fashion-conscious audience.  Use bold text, emojis, and humor where appropriate.  Mention that the new collection is inspired by the attached image.  Include a call to action to visit the website or store.  Keep the email concise.

        """

        response = model.generate_content([email_prompt]) # Pass image too
        email_content = response.text.strip()

        return email_content


    except Exception as e:
        print(f"An error occurred during email generation: {e}")
        return "Error generating email."

In [ ]:
def generate_image_with_text(image_path, prompt):
    """Generates an image with text overlay using Imagen."""


    try:

        image= Image(pil_to_image(image_path))


        print(image)

        # Create SubjectReferenceImage objects
        ref_image_1 = SubjectReferenceImage(
            image=image,
            reference_id=1,
            subject_type="default",
        )



        # Generate the image using the customization model
        generation_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")
        negative_prompt = "disfigured, distorted, mutilated, deformed, grotesque, changes to outfit, alterations to clothing" # Customize as needed

        customization_model = ImageGenerationModel.from_pretrained("imagen-3.0-capability-001")
        images = customization_model._generate_images(
            prompt=prompt,
            number_of_images=4,
            negative_prompt=negative_prompt,
            aspect_ratio="1:1",
            reference_images=[ref_image_1],
            safety_filter_level="block_few",
            person_generation="allow_adult",

        )

        generated_image = images[0]
        # print("Images", images)
        # outputs = [this_image._pil_image for this_image in images]
        # for i in range(4):
        #   if len(outputs) < 4 :
        #     outputs.append(outputs[0])
        #     print("length:" , len(outputs))
        # print("Output",outputs)
        # return outputs
        return generated_image._pil_image

    except Exception as e:
        print(f"An error occurred during email generation: {e}")
        return "Error generating email."

## Website Generation

In [ ]:
def generate_website(image,prompt):
    """Generates a fun and sassy email using Gemini based on the image and prompt."""
    try:
        # image = resize_image(image_path)
        # # new_image_path = pil_to_file(image,"/content/sample_data/image.jpeg")
        image = image_gen.from_bytes(image_to_bytes(image) )

        # # Convert the image to base64
        # image = image_to_base64(image)

        # print("Email Image: " , image)
        model = GenerativeModel("gemini-1.5-flash-002")

        # Gemini prompt for email generation
        website_prompt = f"""
        Write a well managed website post announcing a new collection, based on the uploaded image and the following prompt:
        {prompt}




        The website should be engaging and reflect current fashion trends. Target a young, fashion-conscious audience.  Use bold text, emojis, and humor where appropriate.  Mention that the new collection is inspired by the attached image.  Include a call to action to visit the website or store.

        """

        response = model.generate_content([image,website_prompt]) # Pass image too
        website_content = response.text.strip()

        return website_content


    except Exception as e:
        print(f"An error occurred during email generation: {e}")
        return "Error generating email."

In [ ]:
def generate_image_with_text(image_path, prompt):
    """Generates an image with text overlay using Imagen."""


    try:

        image= Image(pil_to_image(image_path))


        print(image)

        # Create SubjectReferenceImage objects
        ref_image_1 = SubjectReferenceImage(
            image=image,
            reference_id=1,
            subject_type="default",
        )



        # Generate the image using the customization model
        generation_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")
        negative_prompt = "disfigured, distorted, mutilated, deformed, grotesque, changes to outfit, alterations to clothing" # Customize as needed

        customization_model = ImageGenerationModel.from_pretrained("imagen-3.0-capability-001")
        images = customization_model._generate_images(
            prompt=prompt,
            number_of_images=4,
            negative_prompt=negative_prompt,
            aspect_ratio="1:1",
            reference_images=[ref_image_1],
            safety_filter_level="block_few",
            person_generation="allow_adult",

        )

        generated_image = images[0]
        # print("Images", images)
        # outputs = [this_image._pil_image for this_image in images]
        # for i in range(4):
        #   if len(outputs) < 4 :
        #     outputs.append(outputs[0])
        #     print("length:" , len(outputs))
        # print("Output",outputs)
        # return outputs
        return generated_image._pil_image

    except Exception as e:
        print(f"An error occurred during email generation: {e}")
        return "Error generating email."




### UI


In [ ]:
import json
with gr.Blocks() as iface:
    iface.title = "Soch Apparels"

    image_url = "https://images.yourstory.com/cs/images/companies/Soch-1634551344036.jpg?fm=auto&ar=1%3A1&mode=fill&fill=solid&fill-color=fff&format=auto&w=384&q=75"

    gr.HTML(f"""
    <div style="display: flex; justify-content: center; padding-top: 0;">
      <img src="{image_url}" alt="Soch" style="width: 300px; height: auto;"">
    </div>
    """)
    with gr.Row():
        with gr.Column():
            image1_input = gr.Image(type="pil", label="Reference Image 1",width="200px")
            desc1_input = gr.Textbox(label="Description")
            gemini_button1 = gr.Button("Generate Description")
            gemini_button1.click(generate_description, inputs=image1_input, outputs=desc1_input)

        with gr.Column():
            image2_input = gr.Image(type="pil", label="Reference Image 2",width="200px")
            desc2_input = gr.Textbox(label="Description")
            gemini_button2 = gr.Button("Generate Description")
            gemini_button2.click(generate_description, inputs=image2_input, outputs=desc2_input)

        # with gr.Column():
        #     image3_input = gr.Image(type="pil", label="Reference Image 3",width="200px")
        #     desc3_input = gr.Textbox(label="Description")
        #     gemini_button3 = gr.Button("Generate Description")
        #     gemini_button3.click(generate_description, inputs=image3_input, outputs=desc3_input)



    prompt_input = gr.Textbox(label="Prompt")
    refine_button = gr.Button("Refine Prompt")  # New button for prompt refinement
    generate_prompt_button = gr.Button("Generate Prompt") # New button
    generate_image_button = gr.Button("Generate Image")  # Renamed for clarity
    generated_image_output = [gr.Image(type="pil", label="Generated Image") for _ in range(4)]


    # Styling suggestions
    styling_suggestions = gr.Textbox(label="Styling Suggestions", lines=5, visible=False)
    style_button = gr.Button("Product Styling", visible=False)
    styling_suggestions_markdown = gr.HTML("""Styling Suggestions (Markdown)""", visible=False)  # Initially hidden

    # Product Description section (NEW)
    gr.HTML(f"""
    <div>
      <h2 style="text-align: center;">Product Description</h2>
    </div>
    """)
    product_input=gr.Image(type="pil", label="Reference Image 2",width="200px")
    # product_description_output = gr.Textbox(label="Product Description", lines=10, visible=False)  # More lines for the table
    generate_description_button = gr.Button("Generate Product Description")  # New button
    product_description_output = gr.Textbox(label="Product Description", lines=10)


    # style_button = gr.Button("Change to Markdown", visible=False) # Initially hidden
        # styling_suggestions = gr.HTML("""<p>Styling Suggestions</p>""")


    refine_button.click(refine_prompt, inputs=[prompt_input,desc1_input, desc2_input], outputs=prompt_input)


    generate_prompt_button.click(
        generate_prompt,
        inputs=[desc1_input, desc2_input],
        outputs=prompt_input,
    )

    def show_generated_images(*args):
        # Function to generate images and make them visible
        images = generate_image(*args)
        return images + [gr.update(visible=True) for _ in generated_image_outputs]

    generate_image_button.click(  # Click handler for image generation
        generate_image,
        inputs=[image1_input, image2_input, desc1_input, desc2_input,  prompt_input],
        outputs=generated_image_output,
    ).then( #makes the convert to markdown button visible
        fn=lambda x: gr.update(visible=True),
        inputs=styling_suggestions,
        outputs=style_button,
    )

    style_button.click(
        generate_style_and_convert_to_markdown,
        inputs=[desc1_input, desc2_input, prompt_input],
        outputs=[styling_suggestions_markdown, styling_suggestions_markdown ],
    )

    generate_description_button.click(
            generate_product_description,
            inputs=product_input,  # Use the first generated image
            outputs=product_description_output,
        )



     ## Campaign creation
    email_prompt_section_visible = False
    website_prompt_section_visible = False
    social_media_prompt_section_visible = False

      # New section for content generation buttons
    email_button = gr.Button("Generate Email")


    # Email prompt section
    with gr.Row(visible=False) as email_prompt_section:
      with gr.Column():
        image_input = gr.Image(label="Upload an Image", type="pil")
        with gr.Column():
            prompt_input = gr.Textbox(label="Enter Prompt for Email")
            prompt_image_email_imput = gr.Textbox(label="Enter Prompt for Image")
        with gr.Column():
          generate_button = gr.Button("Generate Email")
          generate_image_button = gr.Button("Generate Image ")
      with gr.Column():
        email_output = gr.Textbox(label="Generated Email", lines=10)
        generated_image_output = gr.Image(label="Generated Image", type="pil")


    # Function to make the email prompt section visible
    def show_email_prompt():
        return gr.update(visible=True)

    # Link the email_generate_button to display the prompt section
    email_button.click(
        show_email_prompt,
        inputs=[],
        outputs=email_prompt_section
    )

    # Link the generate button to the generate_email function
    generate_button.click(
        generate_email,
        inputs=[prompt_input],
        outputs=email_output
    )

    generate_image_button.click(
        generate_image_with_text,
        inputs=[image_input,prompt_image_email_imput],
        outputs=generated_image_output
    )



    #website section

    website_button = gr.Button("Generate Website Campaign")
    # social_media_button = gr.Button("Generate Social Media Campaign")

    # Website prompt section
    with gr.Row(visible=False) as website_prompt_section:
        with gr.Column():
          image_input = gr.Image(label="Upload an Image", type="pil")
          with gr.Column():
            prompt_input = gr.Textbox(label="Enter Prompt for Website")
            prompt_image_website_input = gr.Textbox(label="Enter Prompt for Image")
          with gr.Column():
            generate_button = gr.Button("Generate")
            generate_image_button = gr.Button("Generate Image")
        with gr.Column():
          website_output = gr.Textbox(label="Generated Website")
          generated_image_output = gr.Image(label="Generated Image", type="pil")


    # Function to make the website prompt section visible
    def show_website_prompt():
        return gr.update(visible=True)


    # Link the website_generate_button to display the prompt section
    website_button.click(
        show_website_prompt,
        inputs=[],
        outputs=website_prompt_section
    )

    # Link the generate button to the generate_email function
    generate_button.click(
        generate_website,
        inputs=[image_input, prompt_input],
        outputs=website_output
    )

    generate_image_button.click(
        generate_image_with_text,
        inputs=[image_input,prompt_image_website_input],
        outputs=generated_image_output
    )


    # website_button.click(
    #     generate_website_campaign,
    #     inputs=[generated_image_output[0], desc1_input, desc2_input, prompt_input],  # Pass necessary inputs
    #     outputs=website_output,
    # )


    # social_media_button.click(
    #     generate_social_media_campaign,
    #     inputs=[generated_image_output[0], desc1_input, desc2_input, prompt_input],  # Pass necessary inputs
    #     outputs=social_media_output,
    # )





iface.launch(debug=True, show_error=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1ccb3fb4311a73ded3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


 BS  <vertexai.generative_models._generative_models.Image object at 0x7998aacb9550>
 BS  <vertexai.generative_models._generative_models.Image object at 0x7998a88e7850>
<PIL.Image.Image image mode=RGB size=439x698 at 0x7998A7259210> <PIL.Image.Image image mode=RGB size=706x1024 at 0x7998A88EE2D0> That's a lovely image of a woman wearing a light blue, floral-print midi dress. 


Here's a breakdown of what's visible:

* **The Dress:** The dress is the focal point, featuring a light blue base with a smaller, darker blue floral print scattered across it.  The top portion of the dress, from the neckline to just below the bust, has a different, embroidered design in a slightly darker blue shade.  It has three-quarter sleeves and a flared skirt. There's a subtle gold trim at the hem.  The dress appears to be made from a cotton or cotton-blend fabric.

* **The Woman:** The woman is young and appears to be of South Asian descent. She's wearing the dress and has on simple, elegant earrings. Her h